In [1]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import load_metric
# from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.metrics import balanced_accuracy_score

c:\Users\Asus\anaconda3\envs\BDC2024\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv("./dataset/processed dataset/DatasetTrainAllVersion.csv")

dataset.drop_duplicates(inplace=True)

In [3]:
dataset

,text,label
0,kunjungan prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,rt anies dapat tepuk tangan meriah saat jadi r...,Politik
2,@ciqxqwgat04tmtx4ocatxjovq7vv/y8heyaiogmfg8y= ...,Demografi
3,rt @l3r8xfbw3wgbxrpsj0/0hhztbqvgx7qtfwrg9zmhk7...,Politik
4,anies baswedan harap asn termasuk tni dan polr...,Politik
...,...,...
54722,pengundulan hutan di indonesia sat ini menurut...,Sumber Daya Alam
54723,menurut mahfud md calon wakil presiden nomor u...,Sumber Daya Alam
54724,menurut calon wakil presiden nomor urut mahfud...,Sumber Daya Alam
54725,calon wakil presiden nomor urut mahfud md meng...,Sumber Daya Alam


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X = list(dataset['text'].astype(str))
y = list(dataset['label'])

y = le.fit_transform(y)

In [5]:
from sklearn.model_selection import train_test_split

# Memisahkan data menjadi train dan test set
train_texts, eval_texts, train_labels, eval_labels = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

tokenizer = AutoTokenizer.from_pretrained('indolem/indobertweet-base-uncased')

train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
eval_dataset = CustomDataset(eval_texts, eval_labels, tokenizer)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained('indolem/indobertweet-base-uncased', num_labels=8)
model = model.to('cuda')

# Define training arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    balanced_acc = balanced_accuracy_score(labels, predictions)
    return {"balanced_accuracy": balanced_acc}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Asus\anaconda3\envs\BDC2024\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./bert_tweet_50K")
trainer.evaluate()

  0%|          | 0/27365 [00:00<?, ?it/s]c:\Users\Asus\anaconda3\envs\BDC2024\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  2%|▏         | 500/27365 [10:15<9:14:10,  1.24s/it] 

{'loss': 0.8164, 'grad_norm': 3.5057549476623535, 'learning_rate': 4.908642426457154e-05, 'epoch': 0.09}


  4%|▎         | 1000/27365 [20:55<9:30:31,  1.30s/it]

{'loss': 0.5639, 'grad_norm': 3.305156946182251, 'learning_rate': 4.8172848529143066e-05, 'epoch': 0.18}


  5%|▌         | 1500/27365 [31:41<9:15:22,  1.29s/it]

...

  9%|▉         | 2500/27365 [52:16<8:23:48,  1.22s/it]

{'loss': 0.3328, 'grad_norm': 5.650561332702637, 'learning_rate': 4.5432121322857666e-05, 'epoch': 0.46}


 11%|█         | 3000/27365 [1:03:17<8:57:03,  1.32s/it]

{'loss': 0.2752, 'grad_norm': 19.023136138916016, 'learning_rate': 4.45185455874292e-05, 'epoch': 0.55}


 13%|█▎        | 3432/27365 [1:12:52<8:49:57,  1.33s/it]

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.preprocessing import LabelEncoder

# Load dataset
dataset = pd.read_csv("./predict data/raw_text.csv")

# Prepare texts
X = list(dataset['text'].astype(str))

# Define path to the model
model_path = './bert_model/raw_twitbert/raw_tweetbert'

# Check if GPU is available
device = torch.device('cpu')

# Load tokenizer and model from local directory
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

def predict_batch(texts, batch_size=16):
    """Tokenize texts and perform prediction in batches."""
    results = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            results.extend(batch_predictions)

    return results

# Perform batch prediction
batch_size = 16  # Adjust batch size as needed
predicted_labels = predict_batch(X, batch_size)

# Convert predictions to actual class labels
label = ['Politik',
 'Ideologi',
 'Pertahanan dan Keamanan',
 'Sosial Budaya',
 'Ekonomi',
 'Demografi',
 'Geografi',
 'Sumber Daya Alam']

le = LabelEncoder()
le.fit(label)  # Fit label encoder on the dataset labels
predicted_labels = le.inverse_transform(predicted_labels)

# Create a new DataFrame with text and predicted labels
results_df = pd.DataFrame({
    'IDText' : dataset['IDText'],
    'Text': X,
    'Predicted_Label': predicted_labels
})

# Optionally, you can save this DataFrame to a CSV file
results_df.to_csv("./predict data/result/raw_tweet.csv", index=False)

print(results_df.head())  # Print the first few rows of the DataFrame

    IDText                                               Text Predicted_Label
0  TXT0001  lu mau org2 pro-demokrasi di negara ini bisa p...         Politik
1  TXT0002  prabowo ditanya soal hutang luar negeri dia me...         Ekonomi
2  TXT0003  kiki_daliyo  ganjar pranowo itulah beliau soso...        Ideologi
3  TXT0004  @kumparan prabowo gibran yang bisa melakukan i...         Politik
4  TXT0005  @sniperruben45 @uda_zulhendra @ainunnajib lah ...         Politik


In [4]:
test = pd.read_csv("./dataset/raw dataset/dataset_unlabeled_penyisihan_bdc_2024.csv",delimiter=";")

In [8]:
test.to_excel("test_excel.xlsx",index=False)

## FIND MODE

In [55]:
result = pd.read_csv("./Result/Result Temp.csv")

# modus = result[['Predicted_Label_raw_tweet','Predicted_Label_Distilbert','Predicted_Label_BertTweet']].mode(axis=1)

In [61]:
result

,IDText,Text,Predicted_Label_raw_tweet,Predicted_Label_BertTweet,modus
0,TXT0001,lu mau org2 pro-demokrasi di negara ini bisa p...,Politik,Politik,Politik
1,TXT0002,prabowo ditanya soal hutang luar negeri dia me...,Ekonomi,Ekonomi,Ekonomi
2,TXT0003,kiki_daliyo ganjar pranowo itulah beliau soso...,Ideologi,Ideologi,Ideologi
3,TXT0004,@kumparan prabowo gibran yang bisa melakukan i...,Politik,Politik,Politik
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib lah ...,Politik,Politik,Politik
...,...,...,...,...,...
995,TXT0996,"bikin bangga deh, ganjar-mahfud mau alokasikan...",Ekonomi,Politik,Ekonomi
996,TXT0997,pak jokowi sebelum pilpres 2024 berbesar hati ...,Politik,Politik,Politik
997,TXT0998,@datuakrajoangek sbaiknya si gemot nga usah ik...,Sumber Daya Alam,Politik,Politik
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...,Ekonomi,Politik,Ekonomi


In [ ]:
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

def find_mode(row):
    mode = row[['Predicted_Label_raw_tweet', 'Predicted_Label_BertTweet']].mode()
    print(mode)
    if len(mode) == 0:
        return row['Predicted_Label_BertTweet']  # Jika tidak ada modus, kembalikan None atau nilai lain sesuai kebutuhan
    else:
        return mode.iloc[0]  # Mengambil nilai modus dari baris pertama, kolom pertam
    
modus = result.apply(find_mode, axis=1)

result['modus'] = modus

In [67]:
result['modus']

0       Politik
1       Ekonomi
2      Ideologi
3       Politik
4       Politik
         ...   
995     Ekonomi
996     Politik
997     Politik
998     Ekonomi
999     Politik
Name: modus, Length: 1000, dtype: object

In [85]:
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

In [86]:
predict = result['modus']
label = result['Label']

In [53]:
result.to_csv("bismillah_2.csv",index=False)

In [87]:
balanced_acc = balanced_accuracy_score(label, predict)

print("Balanced Accuracy:", balanced_acc)

Balanced Accuracy: 0.5314476525678185


In [96]:
from itertools import combinations
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

best_accuracy = 0.0
best_combination = None
best_prediction = None

# Mendefinisikan kolom target dan fitur
features = ['Predicted_Label_raw_tweet', 'Predicted_Label_Indolem', 'Predicted_Label_Distilbert', 'Predicted_Label_BertTweet']
target = 'Label'

# Menggunakan setiap kombinasi fitur
for r in range(1, len(features) + 1):
    for feature_combination in combinations(features, r):
        selected_features = list(feature_combination)
        
        # Fungsi untuk mencari modus dari kombinasi fitur yang dipilih
        def find_mode(row):
            mode = row[selected_features].mode()
            if len(mode) == 0:
                return row['Predicted_Label_BertTweet']
            else:
                return mode.iloc[0]
        
        # Menghitung modus dan membuat prediksi
        modus = result.apply(find_mode, axis=1)
        result['modus'] = modus
        
        predict = result['modus']
        label = result[target]
        
        # Menghitung balanced accuracy score
        balanced_acc = balanced_accuracy_score(label, predict)
        
        # Memperbarui jika ditemukan yang lebih baik
        if balanced_acc > best_accuracy:
            best_accuracy = balanced_acc
            best_combination = selected_features
            best_prediction = predict  # Menyimpan prediksi terbaik

print("Best Combination of Features:", best_combination)
print("Best Balanced Accuracy:", best_accuracy)
print("Best Prediction:", best_prediction.values)

Best Combination of Features: ['Predicted_Label_raw_tweet', 'Predicted_Label_Indolem']
Best Balanced Accuracy: 0.5660748302988858
Best Prediction: ['Politik' 'Ekonomi' 'Ideologi' 'Politik' 'Politik' 'Politik' 'Politik'
 'Pertahanan dan Keamanan' 'Ideologi' 'Ekonomi' 'Politik' 'Politik'
 'Politik' 'Politik' 'Sosial Budaya' 'Politik' 'Ekonomi' 'Ekonomi'
 'Demografi' 'Politik' 'Politik' 'Politik' 'Ekonomi' 'Geografi' 'Ekonomi'
 'Politik' 'Pertahanan dan Keamanan' 'Politik' 'Ideologi' 'Politik'
 'Politik' 'Politik' 'Politik' 'Politik' 'Politik' 'Politik' 'Politik'
 'Politik' 'Politik' 'Politik' 'Politik' 'Ekonomi' 'Politik' 'Ekonomi'
 'Politik' 'Ekonomi' 'Ekonomi' 'Politik' 'Politik' 'Politik' 'Politik'
 'Politik' 'Politik' 'Politik' 'Politik' 'Demografi' 'Politik' 'Politik'
 'Politik' 'Politik' 'Politik' 'Pertahanan dan Keamanan' 'Politik'
 'Politik' 'Politik' 'Ekonomi' 'Politik' 'Politik' 'Politik'
 'Pertahanan dan Keamanan' 'Politik' 'Politik' 'Politik' 'Politik'
 'Politik' 'Ekonomi' 'P

In [101]:
result['Kelas'] = best_prediction.values

In [102]:
result[['IDText','Kelas']]

,IDText,Kelas
0,TXT0001,Politik
1,TXT0002,Ekonomi
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Politik
...,...,...
995,TXT0996,Ekonomi
996,TXT0997,Politik
997,TXT0998,Sosial Budaya
998,TXT0999,Ekonomi


In [105]:
result[['Text','Kelas']].to_csv("best_answer.csv",index=False)

In [103]:
result['Kelas'].value_counts()

Kelas
Politik                    642
Ekonomi                    150
Pertahanan dan Keamanan     96
Ideologi                    53
Sosial Budaya               23
Demografi                   23
Geografi                     7
Sumber Daya Alam             6
Name: count, dtype: int64